In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd

## Load the FIPS codes for the US states

In [ ]:
county_bounding_boxes_FIPS = pd.read_csv('FIPS/US FIPS Codes.csv')

states_to_drop = ['American Samoa', 'Puerto Rico', 'Alaska', 'Hawaii', 'Commonwealth of the Northern Mariana Islands', 'United States Virgin Islands', 'Guam']

# Columns names in the first row
county_bounding_boxes_FIPS.columns = county_bounding_boxes_FIPS.iloc[0]
county_bounding_boxes_FIPS = county_bounding_boxes_FIPS[1:]
county_bounding_boxes_FIPS = county_bounding_boxes_FIPS.iloc[:, :-1]

county_bounding_boxes_FIPS = county_bounding_boxes_FIPS[~county_bounding_boxes_FIPS['State'].isin(states_to_drop)]

county_bounding_boxes_FIPS.head()

In [ ]:
county_bounding_boxes_FIPS.to_csv('FIPS/US_FIPS_Codes.csv', index=False)

In [ ]:
FIPS = pd.read_csv("/Users/jack/Library/CloudStorage/GoogleDrive-limjackailjk@gmail.com/My Drive/Solar PV Lab/NIMBY Project/Solar NIMBY Final/Solar-NIMBY/county_box/FIPS/US_FIPS_Codes.csv", dtype={'FIPS State': str, 'FIPS County': str})

FIPS

## Read in shapefile of Census bounding boxes to get exact shapely polygons for each county

In [ ]:
county_bounding_boxes_full = gpd.read_file('US County Boundary 2018/cb_2018_us_county_500k.shp', dtype={'STATEFP': str, 'COUNTYFP': str})
county_bounding_boxes_full.head()

## Merge both dataframes on STATEFP

In [ ]:
county_bounding_boxes_full = county_bounding_boxes_full.merge(county_bounding_boxes_FIPS.drop_duplicates(), left_on=['STATEFP', 'COUNTYFP'], right_on=['FIPS State', 'FIPS County'])
county_bounding_boxes_full.head()

## Calculate the area of each county polygon
- epsg 5070 is the projection used by to get the exact area of each county polygon without distortion for the US

In [ ]:
# Calculate the area of each county
county_bounding_boxes_full['area km2'] = county_bounding_boxes_full['geometry'].to_crs(epsg=5070).map(lambda p: p.area / 10**6)
# sort the counties by StateFP and CountyFP
county_bounding_boxes_full = county_bounding_boxes_full.sort_values(['STATEFP', 'COUNTYFP'])
# get sq mile area
county_bounding_boxes_full['area mi2'] = county_bounding_boxes_full['area km2'] * 0.386102
county_bounding_boxes_full.head()

## Get relavant columns and Remove Duplicate counties

In [ ]:
county_bounding_boxes_full = county_bounding_boxes_full[[
    "GEOID", "State", "County Name", "area km2", "area mi2", 'FIPS State', 'FIPS County'
]]


# find all the state county pairs that are duplicated
county_bounding_boxes_full[county_bounding_boxes_full.duplicated(subset=['State', 'County Name'], keep=False)].sort_values(['State', 'County Name'])

county_bounding_boxes_full.head()

In [ ]:
# index of the duplicate cases that are incorrect (checked the area one by one by hand)
idxs = [1703, 1318, 2646, 2617, 2626, 2605]
county_bounding_boxes_full = county_bounding_boxes_full.drop(idxs)
county_bounding_boxes_full.head()

## Save to CSV

In [ ]:
county_bounding_boxes_full.to_csv("county_bounding_boxes_full.csv", index=False)